In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
data = pd.read_csv("myDataFile.csv", low_memory =False)

In [ ]:
items_name = list(data.columns)
items_dict = dict()

for i, it in enumerate(items_name):
    items_dict[it] = i + 1

In [ ]:
trans = list()

for i, r in data.iterrows():
    transaction = set()
    
    for item in items_dict:
        if r[item] == 't':
            transaction.add(items_dict[item])
    trans.append(transaction)
trans

In [ ]:
Candidates = {}
accepted_itemsets = {}
supp_count_L = {}
min_support = 0.005
l = 1
Discarded = {l : []}
Candidates.update({l : [ [f] for f in items_dict.values()]})
Candidates

In [ ]:
def frequency_of_items(itemset, trans):
    count =0
    for i in range(len(trans)):
        if set(itemset).issubset(set(trans[i])):
            count += 1
    return count


def frequent_subsets(itemsets, transactions, min_support, prev_discarded):
    accepted = []
    supp = []
    new_discard = []
    
    k = len(prev_discarded.keys())
    
    for i in range(len(itemsets)):
        flag = pruning(set(itemsets[i]),prev_discarded, k)
        if flag == False:
            count = frequency_of_items(itemsets[i], trans)
            c = count/len(transactions)
            if c >= min_support:
                accepted.append(itemsets[i])
                supp.append(c)
            else:
                new_discard.append(itemsets[i])
    return accepted,supp,new_discard

def self_join(its):
    new_itemsets = []
    for i in range(len(its)):
        for j in range(i+1, len(its)):
            it_join = its[i] + its[j]
            if len(it_join) > 0:
                new_itemsets.append(it_join)
    return new_itemsets

def pruning(item, prev_discarded, k):
    flag = False
    if k > 0:
        for j in prev_discarded[k]:
            if set(j).issubset(item):
                flag = True
                break
    return flag
        

def table(items, support):
    print("Itemsets | Support")
    for k in range(len(items)):
        
        print("{} : {}".format(items[k], support[k]))
        print("\n\n")      

In [ ]:
 ######### Generate L1 #############

f,supp,dis = frequent_subsets(Candidates[l], trans, min_support, Discarded)
Discarded.update({l : dis})
accepted_itemsets.update({l : f})
supp_count_L.update({l: supp})

In [ ]:
k = l + 1
flag = False
while (flag == False):
    Candidates.update({k : self_join(accepted_itemsets[k-1])})
    new_itemsets,support_count,new_discarded = frequent_subsets(Candidates[k], trans, min_support, Discarded)
    Discarded.update({k : new_discarded})
    accepted_itemsets.update({k : new_itemsets})
    supp_count_L.update({ k : support_count})
    print("levels ", k)
    if len(accepted_itemsets[k]) ==0:
        covergence = True
    k +=1

In [ ]:
 table(accepted_itemsets[1], supp_count_L[1])